It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data.
    How do you tidy it up so you can get on with the analysis?  
Take this monstrosity as the DataFrame to use in the following puzzles:  
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',  
'Budapest_PaRis', 'Brussels_londOn'],  
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],  
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],  
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',  
'12. Air France', '"Swiss Air"']}) 

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',  
'Budapest_PaRis', 'Brussels_londOn'],  
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],  
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],  
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',  
'12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [2]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

1.Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [3]:
df['FlightNumber'].interpolate(method ='linear',inplace=True)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [4]:
df['FlightNumber']=df['FlightNumber'].astype('int64',copy=True)
df.dtypes

From_To         object
FlightNumber     int64
RecentDelays    object
Airline         object
dtype: object

2. The From_To column would be better as two separate columns! Split each string on the underscore
delimiter _ to givea new temporary DataFrame with the correct values. Assign the correct column names to 
this temporary DataFrame.

In [5]:
new=df["From_To"].str.partition("_", True)
new.rename(columns={0: "Departure_From", 2: "Arrival_At"},inplace=True)
new

,Departure_From,1,Arrival_At
0,LoNDon,_,paris
1,MAdrid,_,miLAN
2,londON,_,StockhOlm
3,Budapest,_,PaRis
4,Brussels,_,londOn


3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. 
Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".) 

In [6]:
new['Departure_From']=new['Departure_From'].str.capitalize()
new['Arrival_At']=new['Arrival_At'].str.capitalize()
new


,Departure_From,1,Arrival_At
0,London,_,Paris
1,Madrid,_,Milan
2,London,_,Stockholm
3,Budapest,_,Paris
4,Brussels,_,London


In [7]:
new.drop(1,axis=1,inplace=True)

4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions

In [8]:
df.drop("From_To",axis=1,inplace=True)

In [9]:
df_new=pd.concat([df,new],axis=1,copy=True)
df_new

,FlightNumber,RecentDelays,Airline,Departure_From,Arrival_At
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first
value in its own column, each second value in its own column, and so on. 
If there isn't an Nth value, the value should be NaN.  
Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and 
replace the unwanted RecentDelays column in df with delays.   

In [10]:
rd = df['RecentDelays'].apply(pd.Series)
rd = rd.rename(columns = lambda x : 'dealys_' + str(x+1),copy=True)
rd

,dealys_1,dealys_2,dealys_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [11]:
df_new.drop("RecentDelays" ,axis=1,inplace=True)

In [12]:
df_new1=pd.concat([df_new,rd],axis=1,copy=True)
df_new1

,FlightNumber,Airline,Departure_From,Arrival_At,dealys_1,dealys_2,dealys_3
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN


In [13]:
df_new1=df_new1[['FlightNumber','dealys_1','dealys_2','dealys_3','Airline','Departure_From','Arrival_At']]
df_new1

,FlightNumber,dealys_1,dealys_2,dealys_3,Airline,Departure_From,Arrival_At
0,10045,23.0,47.0,NaN,KLM(!),London,Paris
1,10055,NaN,NaN,NaN,<Air France> (12),Madrid,Milan
2,10065,24.0,43.0,87.0,(British Airways. ),London,Stockholm
3,10075,13.0,NaN,NaN,12. Air France,Budapest,Paris
4,10085,67.0,32.0,NaN,"""Swiss Air""",Brussels,London
